---
title: "Graph development"
format:
  html:
    code-fold: true
jupyter: python3
---

In [ ]:
import os 
os.chdir('..')

In [ ]:
import polars as pl
import geopandas as gpd

In [ ]:
roads_df = gpd.read_file("data/interim/roads_2012_06.gpkg", engine="pyogrio")
puma = gpd.read_file("data/interim/puma_06.gpkg", engine="pyogrio")
tmp_puma = puma[(puma["geo_id"] == "0600101")].copy().reset_index(drop=True)
clipped = roads_df.clip(tmp_puma['geometry'])

In [ ]:
base = tmp_puma.plot(color='white', edgecolor='black')
clipped.plot(ax=base, marker='o', color='red', markersize=5)

In [ ]:
clipped.length.sum()

In [ ]:
empty_df = [
  pl.Series("year", [], dtype=pl.Int64),
  pl.Series("puma_id", [], dtype=pl.String),
  pl.Series("leangth", [], dtype=pl.Float64)
]
df = pl.DataFrame(empty_df)
puma2 = puma[puma["geo_id"].str.startswith("06")]
for pum in puma2["geo_id"]:
    tmp = puma.loc[puma["geo_id"] == pum]
    clipped = roads_df.clip(tmp['geometry'])
    leng = pl.DataFrame(
      {
        "year": 2012,
        "puma_id": pum,
        "leangth": clipped.length.sum()
      }
    )
    df = pl.concat([df, leng], how="vertical")
print(df)

In [ ]:
def process(roads, state_id, pumas_df):
  empty_df = [
              pl.Series("year", [], dtype=pl.Int64),
              pl.Series("puma_id", [], dtype=pl.String),
              pl.Series("leangth", [], dtype=pl.Float64)
             ]
  df = pl.DataFrame(empty_df)
  pumas = pumas_df[pumas_df["geo_id"].str.startswith(state_id)]
  for puma in pumas["geo_id"]:
      tmp = pumas.loc[pumas["geo_id"] == puma]
      clipped = roads.clip(tmp['geometry'])
      leng = pl.DataFrame(
        {
          "year": 2012,
          "puma_id": puma,
          "leangth": clipped.length.sum()
        }
      )
      df = pl.concat([df, leng], how="vertical")
  return df